## Inisiasi Koneksi Database

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'practicum_student',       
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'}      

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Query Data

**Hitung jumlah buku yang dirilis setelah tanggal 1 Januari 2000.󠀲󠀡󠀠󠀦󠀥󠀩󠀠󠀩󠀳**


In [3]:
query = """
SELECT COUNT(book_id) AS Jumlah_Buku
FROM books
"""
df_1 = pd.io.sql.read_sql(query, con = engine)
df_1

,jumlah_buku
0,1000


In [4]:
query = """
SELECT COUNT(book_id) AS Jumlah_Buku
FROM books
WHERE DATE(publication_date) > '2000-01-01'
"""
df_2 = pd.io.sql.read_sql(query, con = engine)
df_2

,jumlah_buku
0,819


In [5]:
df_3 = df_2/df_1 * 100
df_3.columns = ['Persentase (%) Buku Setelah 1 Januari 2000']
df_3

,Persentase (%) Buku Setelah 1 Januari 2000
0,81.9


Setelah 1 Januari 2000, terdapat 81.9% buku dari keseluruhan buku yang ada di database atau 819 buku dari 1000 total buku.

**Hitung jumlah ulasan pengguna dan rating rata-rata untuk setiap buku.󠀲󠀡󠀠󠀦󠀥󠀩󠀡󠀠󠀳**

In [6]:
query = """
SELECT 
    rating_book.title, rating, COUNT(DISTINCT reviews.review_id) as review_count
FROM 
    reviews
RIGHT JOIN 
    (SELECT books.book_id, books.title, AVG(ratings.rating) as rating
    FROM books
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY books.book_id, books.title) as rating_book ON rating_book.book_id = reviews.book_id
GROUP BY 
    rating_book.title, rating_book.rating
ORDER BY 
    review_count DESC, rating DESC
LIMIT 10;
"""
rating_review_book = pd.io.sql.read_sql(query, con = engine)
rating_review_book

,title,rating,review_count
0,Twilight (Twilight #1),3.662500,7
1,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,The Book Thief,4.264151,6
4,The Glass Castle,4.206897,6
5,Outlander (Outlander #1),4.125000,6
6,The Hobbit or There and Back Again,4.125000,6
7,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
9,Water for Elephants,3.977273,6


Dari sini, dapat diketahui bahwa buku dengan jumlah ulasan yang cukup banyak dengan rating yang tinggi adalah Twilight dengan 7 review dan dengan rating 3.66, disusul dengan seri Harry Potter dengan rating yang sangat tinggi, sekitar 4.41 dan 4.28.

**Identifikasi penerbit yang telah menerbitkan jumlah buku terbanyak dengan jumlah halaman lebih dari 50 (hal ini akan membantu mengecualikan brosur dan publikasi serupa dari analisis).󠀲󠀡󠀠󠀦󠀥󠀩󠀡󠀡󠀳**

In [7]:
query = """

SELECT 
    publishers.publisher_id AS publisher_id, publishers.publisher AS publisher, COUNT(books.book_id) AS book_count
FROM 
    publishers
RIGHT JOIN books ON publishers.publisher_id = books.publisher_id
WHERE 
    books.num_pages > 50
GROUP BY 
    publishers.publisher_id, publisher
ORDER BY 
    book_count DESC
LIMIT 10;

"""
rating_review_book = pd.io.sql.read_sql(query, con = engine)
rating_review_book

,publisher_id,publisher,book_count
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,284,St. Martin's Press,14
8,46,Berkley Books,14
9,83,Delta,13


Penguin Books menempati posisi dengan jumlah buku terbit terbanyak dengan total 42 buku yang telah di publish, disusul oleh Vintage dan Grand Central Publishing dengan jumlah buku masing-masing 31 dan 25.

**Identifikasi penulis dengan rating rata-rata buku tertinggi: temukan buku dengan rating minimal 50.󠀲󠀡󠀠󠀦󠀥󠀩󠀡󠀢󠀳**

In [8]:
query = """
SELECT 
    author_book.author AS Author, AVG(ratings.rating) AS Rating
FROM 
    ratings
RIGHT JOIN 
    (SELECT authors.author_id, authors.author, books.title, books.book_id
    FROM authors
    RIGHT JOIN books ON books.author_id = authors.author_id
    ORDER BY authors.author_id) author_book ON author_book.book_id = ratings.book_id
GROUP BY 
    author_book.author
HAVING 
    COUNT(ratings.username) > 50
ORDER BY 
    Rating DESC
LIMIT 10;
"""
rating_author_book = pd.io.sql.read_sql(query, con = engine)
rating_author_book

,author,rating
0,J.K. Rowling/Mary GrandPré,4.288462
1,Agatha Christie,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,J.R.R. Tolkien,4.240964
4,Roald Dahl/Quentin Blake,4.209677
5,Louisa May Alcott,4.203704
6,Rick Riordan,4.130952
7,Arthur Golden,4.107143
8,Stephen King,4.009434
9,John Grisham,3.971429


Author dengan rating tertinggi dengan jumlah rating terbanyak adalah J.K. Rowling/Mary GrandPré	yang terkenal dengan buku serinya yang berjudul Harry Potter, diikuti dengan Agatha Christie, Markus Zusak, dan J.R.R Tolkien.

**Hitung jumlah rata-rata teks ulasan di antara pengguna yang memberi rating terhadap lebih dari 50 buku.**

In [9]:
query = """
SELECT 
    AVG(reviews_3.review_count) AS AVG_text_ulasan
FROM
    (SELECT ratings.username as username, COUNT(ratings.username) as rating_count, reviews_2.review_count
    FROM ratings
    LEFT JOIN (
    SELECT username, COUNT(text) as review_count
    FROM reviews
    GROUP BY username
    ORDER BY COUNT(username)) as reviews_2 ON reviews_2.username = ratings.username
    GROUP BY ratings.username, reviews_2.review_count
    HAVING COUNT(ratings.username) > 50
    ORDER BY rating_count DESC) AS reviews_3
"""
rating_review = pd.io.sql.read_sql(query, con = engine)
rating_review

,avg_text_ulasan
0,24.333333


Jumlah rata-rata text ulasan sekitar 24.3 dengan rincian sebagai berikut:

In [10]:
query = """

SELECT 
    ratings.username as username, COUNT(ratings.username) as rating_count, reviews_2.review_count
FROM 
    ratings
LEFT JOIN (
    SELECT username, COUNT(text) as review_count
    FROM reviews
    GROUP BY username
    ORDER BY COUNT(username)) as reviews_2 ON reviews_2.username = ratings.username
GROUP BY 
    ratings.username, reviews_2.review_count
HAVING 
    COUNT(ratings.username) > 50
ORDER BY 
    rating_count DESC

"""
rating_review = pd.io.sql.read_sql(query, con = engine)
rating_review

,username,rating_count,review_count
0,martinadam,56,27
1,paul88,56,22
2,richard89,55,26
3,sfitzgerald,55,28
4,jennifermiller,53,25
5,xdavis,51,18


Jumlang keaktifan tertinggi adalah 56 rating dengan jumlah review sebanyak 27.